# Homework 1 - Streaming (15 pts)

### DUE: 02/18/2020 at 5:30pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_NetID.ipynb.

## Task 1 (8 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [6]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
count = 0
for row in csvRows('taxi_events.csv'):
    if row['event'] == 'pickup':
        count = count+1
        if maxHiredCabs<count:
            maxHiredCabs = count
    else:
        count = count - 1
        
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

For the incoming stream of data, count gives the current number of hired cabs. 

Whenever a taxi picks up a person, that is , event = pickup then the variable count gets incremented by 1. 

MaxHiredCabs store the maximum number of hired cabs so whenever maxHiredCabs < count we do maxHiredCabs = count. 

Whenever the event is not pickup but dropoff then the count gets decremented by 1, that is, one of the cabs has completed its trip so the number of current hired cabs (count) reduces by 1. 

Finally, MaxHiredCabs which is the maximum number of hired taxi cabs = 250.

## Task 2 (7 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [8]:
import csv
import dateutil

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
dropoff_list = []

for row in csvRows('taxi_trips.csv'):
    pickup_time = dateutil.parser.parse(row['pickup_time'])
    dropoff_time = dateutil.parser.parse(row['dropoff_time'])
    dropoff_list.append(dropoff_time)
    dropoff_list = [l for l in dropoff_list if l > pickup_time]
    
    if maxHiredCabs < len(dropoff_list):
        maxHiredCabs = len(dropoff_list)
    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

For the incoming stream of data, first we parse the pickup time and dropoff time of the current vehicle and store it in variables pickup_time and dropoff_time respectively. 

We have a list dropoff_list which contains a list of all dropoff times of all data till now. We append the dropoff_time in it. 

We update this dropoff_list now by removing those dropoff times values which is lesser than the current row's pickup time. This means that the trips which have completed before the current trip's pickup time will be removed from the dropoff_list.

Hence, len(dropoff_list) gives us the current number of hired cabs. maxHiredCabs gives the maximum number of hired taxi cabs. If maxHiredCabs < len(dropoff_list) we do maxHiredCabs=len(dropoff_list). Finally we get maximum number of hired taxi cabs which is 250.